# Fourier Series Visualization Breakdown
## Imports

I am importing all necessary packages for this to function properly.

In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import matplotlib.animation as animate
import numpy as np
import cv2 as cv
from scipy import integrate
from numpy import pi

## User Inputs

Here we have user inputs that will affect the speed, length, and percision of the animation

    T - The period of the animation: How long it will take to complete the animation
    N - The order of the Fourier series used to approximate the input (the higher the better)
    FPS - How many frames each second of the animation will contain

In [2]:
## User Inputs ##
T = 20  # period
N = 4  # order
FPS = 60  # fps of video

## Image Conversion

Here we are converting the input image to black and white and then using that to get the outline of the input. Once this is done, the console will output "Image processed". If you download this and are trying it out yourself, replace 'woman.jpg' with the image you are trying to create the animation of! (Make sure they're in the same file path)

In [3]:
## Converts Image to grayscale and provides xlist and ylist ##
# Converting into grayscale
im = cv.imread('woman.jpg')
imgray = cv.cvtColor(im, cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(imgray, 127, 255, 0)
contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
cnt = 0
for i in range(len(contours)):
    if len(contours[i]) > len(contours[cnt]):
        cnt = i
print("Image processed")

Image processed


Now, we are splitting the list of points defining the outline of the image into x and y components and recentering the image at (0, 0). Finally, we define x(t) and y(t). This is because this image is defined discretely through the array of points in xlist and ylist, or x[n] and y[n], but through interpolation, we are able to have a continuous function for x(t) and y(t) where the values  are along the lines connecting the points of x[n] and y[n].

In [4]:
# Gets contour into x and y components
xlist = np.array(contours[cnt][:, :, 0]).flatten()
ylist = -np.array(contours[cnt][:, :, 1]).flatten()

# Centers image at (0, 0)
xlist = xlist - (np.max(xlist) + np.min(xlist)) / 2
ylist = ylist - (np.max(ylist) + np.min(ylist)) / 2

## x[n], y[n] => x(t), y(t) ##
# Allows for interpolation of contours for a continours function and aliasing (non-integer index, short period/low FPS)
def x(t):
    n = t / T * len(xlist)
    return np.interp(n, range(len(xlist)), xlist, period=len(xlist))


def y(t):
    n = t / T * len(ylist)
    return np.interp(n, range(len(ylist)), ylist, period=len(xlist))

## Plotting the Real Image

Now, we get on to the plotting. In order to animate the dot along the real plot of f(t), the function of the contour described by x(t) and y(t), we need to find which descrete values of f(t) we need in order to create the frames. To do this, an array for the values of t was created from 0 to T with an FPS amount of evenly spaced values between each integer. Since f(t) outputs two real number quantities for x and y coordinates, I decided to map the y coordinates to the complex number line which retains the image in the complex plane, but allows for one output for f(t) allowing f(t) to be approximated with one Fourier series.

$$
    

In [5]:
## f[n] => f(t) ##
# Creating f[n] as a complex exponentional
f_n = xlist + 1j * ylist

# Allows for interpolation of contours for a continours function and aliasing (non-integer index, short period/low FPS)
def f(t):
    n = t / T * len(xlist)
    return np.interp(n, range(len(xlist)), f_n, period=len(xlist))

## Plotting Real Image ##
# Creates common frames
time = np.linspace(0, T, T * FPS)

Now, the first figure, or window, is created that will contain a 3d plot of the contours with an dot animating which point it is along during that current moment in time. The function used to create this animation will update the location of the dot on the contours at time t.

In [6]:
# Create figure and first subplot
fig1 = plt.figure(1)

# Plots image and time marker
ax1 = fig1.add_subplot(111, projection = '3d')
ax1.plot(x(time), y(time), time)
drawn_real_point = ax1.plot(
    [x(0)], [y(0)], [0], color=(1, 49 / 255, 49 / 255, 0.83), marker="."
)[0]

# Shows f(t), the real map of the contour over time
def update_real_point(t):
    drawn_real_point.set_data_3d(x(t), y(t), t)

## Fourier Series Coefficients

In order to estimate f(t) via a Fourier Series, we must find the fourier coefficients for f(t), which is the average value of the amplitude and phase for frequency k for one period of f(t), or

$$
    c_k = \frac{1}{T}\int_{0}^{T}f(t)e^{-j\frac{{2\pi}kt}{T}}dt
$$

Coefficients are added for each discrete frequency, k, from -N to N, where k is the frequency. The frequency is based off of the period, instead of just seconds, so if something has a period of 4, k = 1 would be 1/4 Hz or 1 revolution per period.

In [ ]:
coefficients = []  # array to store coefficients
for n in range(-N, N + 1):  # orders in c₀, c₁, c₋₁, ... cₙ, c_₋ₙ
    integrand = lambda t: f(t) * np.exp(-1j * (2 * pi / T) * t * n)
    coefficients.append(
        (1 / T) * integrate.quad_vec(integrand, 0, T, limit=500, full_output=True)[0]
    )
    # def integrand_real(t): return np.real(f(t) * np.exp(-1j*(2*pi/T)*t*n))
    # def integrand_imag(t): return np.imag(f(t) * np.exp(-1j*(2*pi/T)*t*n))
    # coefficients.append(
    #     (1/T)*integrate.quad(integrand_real, 0, T, limit=100, full_output=True)[0] +
    #     (1j/T)*integrate.quad(integrand_imag, 0, T, limit=100, full_output=True)[0])
    print(
        str(round(((len(coefficients) - 1) / (2 * N)) * 100, 3))
        + "% Done with coefficients..."
    )

0.0% Done with coefficients...
12.5% Done with coefficients...
25.0% Done with coefficients...
37.5% Done with coefficients...
50.0% Done with coefficients...
62.5% Done with coefficients...
75.0% Done with coefficients...
87.5% Done with coefficients...
100.0% Done with coefficients...


## Plotting the complex sinusoids used to approximate f(t)

In order to plot Re{approximation{f(t)}} and Im{approximation{f(t)}}} as their component sinusoids on to planes, another figure was created, though with two subplots this time. The amplitudes and phase-shifts from the fourier coefficients array were used to define the sinusoidal components of f(t). The more red and opaque the sinusoid, the higher the frequency, and the more blue and transparent, the lower the frequency. Finally, one blue sinusoid is drawn as the sum, or the Fourier series approximation, of these sinusoids. These do not seem to resemble f(t), but that is because it describes the x-component of the approximation over time on the left and the y-component on the right. Since f(t) is now described as a single complex quantity, the x-component you see is Re{approximation{f(t)}} and the y-component is Im{approximation{f(t)}}}.

In [8]:
## f(t) => Re{f(t)}, Im{f(t)}
# Plots for X and Y components of Fourier series are made
fig2 = plt.figure(2)
ax2 = fig2.add_subplot(121)
ax3 = fig2.add_subplot(122)

# Creates X components, Y components, and resultant sums
fs_xlist, fs_ylist = [], []
sum_x, sum_y = 0, 0
for n in range(-N, N + 1):
    r = np.absolute(coefficients[n + N])
    phi = np.angle(coefficients[n + N])
    sum_x += r * np.cos((2 * pi * n / T) * time + phi)
    sum_y += r * np.sin((2 * pi * n / T) * time + phi)
    fs_xlist.append(
        ax2.plot(
            time,
            r * np.cos((2 * pi * n / T) * time + phi),
            color=((np.abs(n / N), 0, 1 - np.abs(n / N),.25 + 0.25 * np.abs(n / N))),
        )[0]
    )
    fs_ylist.append(
        ax3.plot(
            time,
            r * np.sin((2 * pi * n / T) * time + phi),
            color=((np.abs(n / N), 0, 1 - np.abs(n / N), .25 + 0.25 * np.abs(n / N))),
        )[0]
    )

On top of the sinusoids, a vertical line is drawn to signify the current x and y component of the approximation for f(t).

In [8]:
# Creates vertical line to track F⁻¹{F{f(t)}} (Approximation of f(t) via our Fourier Series)
time_line_1, time_line_2 = ax2.axvline(x=0), ax3.axvline(x=0)


def update_time_line(t):
    time_line_1.set_xdata([t, t])
    time_line_2.set_xdata([t, t])
    return time_line_1, time_line_2


# Sets components' view
ax2.plot(time, sum_x)
ax2.grid(False)
ax2.set_xticks([])
ax2.set_yticks([])

ax3.plot(time, sum_y)
ax3.grid(False)
ax3.set_xticks([])
ax3.set_yticks([])

[]

## Approximating f(t) via a Fourier Series

Now, the fun part. Since we have the coefficients, we can dive into creating the animation of the approximation. To start, we create 3 more figures: one to demonstrate the output overlayed by the Fourier series, one to show only the output of the Fourier series, and one to map the error of the approximation versus the real definition of the contours. Along with this, an array containing the spirals, or complex sinusoids, drawn for one period is created as well as an array containing the lines to more clearly show the current value of each complex sinusoid as from a bird's eye view, it looks like only circles. Moreover, other objects are created to track the output point of the Fourier series on both graphs, as well as to store the error line. As with the x-component and y-components drawn above, the more red and opaque the sinusoid, the higher the frequency, and the more blue and transparent, the lower the frequency.

In [9]:
## Approximation of f(t) via our Fourier Series
fig5 = plt.figure(5)
ax5 = fig5.add_subplot(121, projection = '3d')
spirals = [
    ax5.plot(
        [],
        [],
        [],
        linestyle="solid",
        color=(
            (0.5 * np.abs(i / N), 0, 1 - 0.5 * np.abs(i / N), .25 + 0.25 * np.abs(i / N))
        ),
        linewidth=1 - 0.35 * i / N,
    )[0]
    for i in range(2 * N + 1)
]  # stores spirals drawn each frame
lines = [
    ax5.plot(
        [],
        [],
        [],
        linestyle="solid",
        color=((0.5 * np.abs(i / N), 0, 1 - 0.5 * np.abs(i / N), .25 + 0.25 * np.abs(i / N))),
        linewidth=1 - 0.35 * i / N,
    )[0]
    for i in range(2 * N + 1)
]  # stores lines drawn each frame
drawn_fs_point = ax5.plot([], [], [], color=(0, 1, 0, 0.5), marker=".")[
    0
]  # Draws current point
drawn_x, drawn_y, drawn_z = [], [], []  # Stores points of drawn points over time
drawn_fs = ax5.plot([], [], [], color=(0, 0, 0, 0.5))[
    0
]  # Plot of drawn points over time

ax6 = fig5.add_subplot(122, projection = '3d')  # Creates plot for only Fourier Series Approximation
fs_only = ax6.plot([], [], [])[0]  # Stores points of Fourier
fs_only_point = ax6.plot([], [], [], color=(0, 1, 0, 0.5), marker=".")[
    0
]  # Draws only current point

# Error Plotting Prep
current_x, current_y, = 0, 0
fig4 = plt.figure(4)
ax4 = fig4.add_subplot(111)
ax4.set_xlim(-T / 2, T + T / 2)
ax4.set_ylim(-1, 5)
error = []
error_plot = ax4.plot([], [])[0]
zero_line = ax4.plot(
    time, np.zeros(len(time)), color=(0, 0, 0, 0.5), linestyle="dashed"  # 0% error line
)

one_line = ax4.plot(
    time, np.ones(len(time)), color=(1, 0, 0, 0.5), linestyle="dashed"
)  # 100% error line

The definition of approximating a function via a Fourier series is
$$
    f(t) = \lim_{N\to\infty}\sum_{k= -N}^{N}c_ke^{-j\frac{{2\pi}kt}{T}}
$$
Therefore, the approximation will never be perfect, but since the order is N, so the higher the order, the closer the approximation. Below, for every frame, the location of each sinusoid is updated, as each sinusoid shifts between frames, the sum of their individual outputs maps to a new point. At the very end of the series, the sinusoid of frequency -N, the last one, is the last to be summed in the series, so the tip of the output of that sinusoid is the Fourier series approximation of f(t). The reason k = -N is the last sinusoid to be drawn is because the sinusoids are drawn from frequency 0 to 1 to -1 to 2 to -2 ... to N to -N. This is because sinusoids rotating in opposite directions with the same frequency will be more likely to keep the visualization of the approximation within the frame.

In [10]:
# Draws Fourier Series, Approximation, and Error
def draw_f(t):
    current_x, current_y = np.real(coefficients[N]), np.real(
        coefficients[N]
    )  # Offset of c₀
    # Draws spirals, lines, drawing, current point, and error in each frame
    for n in range(1, N + 1):
        current = coefficients[N + n] * np.exp(
            1j * (2 * pi / T) * t * n
        )  # calculates current term
        r = np.absolute(current)  # amplitude of complex sinusoid (spiral)
        phi = np.angle(current)  # phase shift of complex sinusoid (spiral)
        # plots entire period of complex sinusoid
        spirals[2 * n - 1].set_data_3d(
            current_x + r * np.cos((2 * pi * n / T) * time + phi),
            current_y + r * np.sin((2 * pi * n / T) * time + phi),
            time,
        )
        # plots line only at the current time
        lines[2 * n - 1].set_data_3d(
            [current_x, current_x + np.real(current)],
            [current_y, current_y + np.imag(current)],
            [t, t],
        )
        current_x, current_y = current_x + np.real(current), current_y + np.imag(
            current
        )
        current = coefficients[N - n] * np.exp(-1j * (2 * pi / T) * t * n)
        r = np.absolute(current)
        phi = np.angle(current)
        spirals[2 * n].set_data_3d(
            current_x + r * np.cos((2 * pi * -n / T) * time + phi),
            current_y + r * np.sin((2 * pi * -n / T) * time + phi),
            time,
        )
        lines[2 * n].set_data_3d(
            [current_x, current_x + np.real(current)],
            [current_y, current_y + np.imag(current)],
            [t, t],
        )
        current_x, current_y = current_x + np.real(current), current_y + np.imag(
            current
        )
    # After the final point is calculated (the sum of the Fourier series approximation at time t)
    # the image is traced and the current point is drawn as well as the
    drawn_fs_point.set_data_3d(current_x, current_y, t)
    fs_only_point.set_data_3d(current_x, current_y, t)
    drawn_x.append(current_x)
    drawn_y.append(current_y)
    drawn_z.append(t)
    drawn_fs.set_data_3d(drawn_x, drawn_y, drawn_z)
    fs_only.set_data_3d(drawn_x, drawn_y, drawn_z)
    return ax5

def draw_error(t):
    error.append(np.abs((f(t) - (current_x + 1j * current_y)) / f(t)))
    error_plot.set_data(drawn_z, error)
    return error_plot

## Graph Tidying

Finally, the graphs are tidied up: all the axes and tick marks are removed, windows are properly sized, gridlines are removed, the 3d graphs are spawned with a bird's eye view to clearly show the image and "circles" the complex sinusoids compose on the x-y plane, and labels are added to the graphs.

In [11]:
# Graph views are set for FS App, FS Drawing, and Error
ax1.view_init(90, -90)
ax1.grid(False)
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_zticks([])

ax5.view_init(90, -90)
ax5.axes.set_xlim3d(left=-max(xlist) - 200, right=max(xlist) + 200)
ax5.axes.set_ylim3d(bottom=-max(ylist) - 200, top=max(ylist) + 200)
ax5.axes.set_zlim3d(bottom=-T / 2, top=T + T / 2)


ax6.view_init(90, -90)
ax6.axes.set_xlim3d(left=np.min(xlist) - 200, right=np.max(xlist) + 200)
ax6.axes.set_ylim3d(bottom=np.min(ylist) - 200, top=np.max(ylist) + 200)
ax6.axes.set_zlim3d(bottom=-T / 2, top=T + T / 2)

ax4.grid(False)
ax4.set_xticks([])
ax4.set_yticks([])

ax5.grid(False)
ax5.set_xticks([])
ax5.set_yticks([])
ax5.set_zticks([])

ax6.grid(False)
ax6.set_xticks([])
ax6.set_yticks([])
ax6.set_zticks([])

## Plot tidying ##
# Shows all plots
ax1.set_facecolor((0, 0, 0, 0))
ax2.set_facecolor((0, 0, 0, 0))
ax3.set_facecolor((0, 0, 0, 0))
ax4.set_facecolor((0, 0, 0, 0))
ax5.set_facecolor((0, 0, 0, 0))
ax6.set_facecolor((0, 0, 0, 0))

ax1.set_title("Contour of Original Image")
ax1.set_xlabel("x position of point")
ax1.set_ylabel("y position of point")
ax1.set_zlabel("Time (sec)")

ax2.set_title("X-components of Fourier Series")
ax2.set_xlabel("Time (sec)")
ax2.set_ylabel("Amplitude of X Component")

ax3.set_title("Y-components of Fourier Series")
ax3.set_xlabel("Time (sec)")
ax3.set_ylabel("Amplitude of Y Component")

ax4.set_title("%" + "Error of Fourier Series vs Original")
ax5.set_xlabel("Time (sec)")
ax5.set_ylabel("%" + "Error (Red = 100%, Grey = 0%)")

ax5.set_title("Fourier Series Approximation of Contours")
ax5.set_xlabel("x axis")
ax5.set_ylabel("y axis")
ax5.set_zlabel("Time (sec)")

ax6.set_title("Output of Fourier Series")
ax6.set_xlabel("x position of point")
ax6.set_ylabel("y position of point")
ax6.set_zlabel("Time (sec)")

Text(0.5, 0, 'Time (sec)')

## Plotting
Now, function animation objects are constructed to update each frame according to time t. When the plot command is run, the graphs will appear animated as it updates with each frame.

In [12]:
## Animation
# Create animation function ojects
real_animation = animate.FuncAnimation(fig1, update_real_point, frames=time)
component_animation = animate.FuncAnimation(fig2, update_time_line, frames=time)
fs_animation = animate.FuncAnimation(fig5, draw_f, frames=time)
error_animation = animate.FuncAnimation(fig4, draw_error, frames=time)

plt.show()